<h1 style="display: flex; align-items: center; font-size: 35px">
    <img src="../assets/icons/icons8-clickteam-fusion-48.png" style="width: 50px; height: 50px; margin-right: 10px;">
    <strong>MRI Brain Alzheimer Classification - Model Ensemble</strong>
</h1>

****

<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-git-48.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Clone Repository from GitHub
</h1>


In [1]:
# Clone repository from GitHub
# !git clone https://github.com/nivbartov/MRI_Brain_Alzheimer_Classification

# %cd MRI_Brain_Alzheimer_Classification
%cd ..

c:\Dor\TECHNION\deep_learning\Project\MRI_Brain_Alzheimer_Classification


<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-import-64.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Import Packages
</h1>


In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import sys
import os
import numpy as np
import pandas as pd
import torch
from torchsummary import summary
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Subset, ConcatDataset
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from kornia import augmentation as K
from kornia.augmentation import AugmentationSequential
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import random
import time
from datetime import datetime
import json
import gc
import subprocess

<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-function-80.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Import Internal Functions
</h1>


In [3]:
from models import def_models
from utils import utils_funcs
from utils import optuna_search

In [4]:
print(f'torch.cuda.is_available(): {torch.cuda.is_available()}')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')
torch.cuda.empty_cache()
gc.collect()
utils_funcs.open_nvitop()

torch.cuda.is_available(): True
device: cuda:0


<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-load-50.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Load Dataset
</h1>

In [5]:
test_set = torch.load('dataset/dataset_variables/test_set.pt')

<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-data-recovery-40.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Models Definition and Initialization
</h1>

**DINOv2**

In [6]:
# Initialize input and output sizes
input_width = 224
input_height = 224
input_channels = 3
output_channels = 4

dino_v2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

# Freeze DINOv2 layers
for param in dino_v2_model.parameters():
    param.requires_grad = False

# Create DINOv2 model with FC layers before the output
dino_v2_model = def_models.DINOv2(DINOv2_backbone=dino_v2_model, output_channels=output_channels).to(device)

pretrained_model_path = f"./checkpoints/DINOv2_074712_09102024/DINOv2_140224_09102024_train_0.0121_val_0.0937.pth"
checkpoint = torch.load(pretrained_model_path,weights_only=True)
dino_v2_model.load_state_dict(checkpoint['net'])

Using cache found in C:\Users\97252/.cache\torch\hub\facebookresearch_dinov2_main


<All keys matched successfully>

**EfficientnetB0**

In [7]:
# Initialize input and output sizes
input_width = 224
input_height = 224
input_channels = 3
output_channels = 4

# Load a pre-trained EfficientNet-B0 model
EfficientNet_backbone_model = torchvision.models.efficientnet_b0(pretrained=True)

# Freeze EfficientNet-B0 layers
for param in EfficientNet_backbone_model.parameters():
    param.requires_grad = True

# Create EfficientNet-B4 model with FC layers before the output
efficientnet_model = def_models.EfficientNet(EfficientNet_backbone=EfficientNet_backbone_model, output_channels=output_channels).to(device)

pretrained_model_path = f"./checkpoints/EfficientNet_083104_17102024/EfficientNet_092843_17102024_train_0.0044_val_0.0401.pth"
checkpoint = torch.load(pretrained_model_path,weights_only=True)
efficientnet_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

**Resnet**

<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-loader-80.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Create Data Loaders
</h1>

In [23]:
batch_size = 64
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


<h1 style="display: flex; align-items: center; font-size: 25px; margin: 0;">
    <img src="../assets/icons/icons8-unite-64.png" style="width: 40px; height: 40px; margin-right: 5px;">
    Model Ensemble
</h1>

**Define parameters**

In [24]:
# General parameters
models_group = [dino_v2_model, efficientnet_model]
weights = [0.2,0.8]

# FGSM parameters
epsilon = 0.005

# PGD parameters
alpha = 0.01
num_iter = 5
pgd_epsilon = 0.005

**Make the Prediction**

In [25]:
ensemble_accuracy = utils_funcs.calculate_ensemble_accuracy(models_group, weights, testloader, device)
print("Ensemble Model Accuracy on Natural Images: {:.3f}%".format(ensemble_accuracy))

Ensemble Model Accuracy on Natural Images: 99.140%
